 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/05/05.ipynb#Načítání-dat-ze-souboru" data-toc-modified-id="Načítání-dat-ze-souboru-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Načítání dat ze souboru</a></span></li><li><span><a href="http://localhost:8888/notebooks/05/05.ipynb#Funkce-apply,-lapply-a-sapply" data-toc-modified-id="Funkce-apply,-lapply-a-sapply-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Funkce <code>apply</code>, <code>lapply</code> a <code>sapply</code></a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/05/05.ipynb#apply" data-toc-modified-id="apply-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><code>apply</code></a></span></li></ul></li></ul></div>

# Manipulace dat a popisná statistika

V předchozích dvou lekcích jsme si ukázali [základní principy programování v R](../03/) a [hlavní datové struktury](../04/). Nyní si ukážeme jak tyto dvě lekce propojit a využít tyto znalosti k jednoduché analýze různých datových souborů. Využijeme k tomu nejrůznějších principů popisné statistiky a ukážeme si též možnosti základní datové vizualizace pro jednoduchou exploraci dat.

## Načítání dat ze souboru

Data většinou nebudete do tabulek a matic vkládat manuálně ale z externích zdrojů, kterými mohou být internetové stránky, nejrůznější databáze nebo obyčejné textové soubory. V R existuje mnoho možností, kterými lze data snadno získat z mnoha zdrojů. My jsme si zatím ukázali jak jednoduše přistupovat k testovacím datovým množinám, které jsou v R přístupné přímo jako proměnné (např. `volcano` nebo `mtcars`). Nejčastěji ale budete jako uživatel chtít načíst do R vlastní data a to ve formě textových souborů. V této lekci si tedy ukážeme jak na to.

V adresáři s touto lekcí se nachází dva soubory `ufocase.info` a `ufocase.bat`. Soubor `ufocase.info` obsahuje popis jedné z [volně dostupných datových množin](http://www.stat.ufl.edu/~winner/datasets.html) na stránkách Larryho Winnera z University of Florida. Tento soubor vypadá následovně:

```
Dataset:  ufocase.dat

Source:  www.ufocasebook.com

Description:  UFO/Alien Cases, including categorical outcomes:
Presence/Absence of physical effects, multimedia (picture/video),
ET contact, and abduction.

Variable/Columns

Year 1-4 
Name of sighting/event  7-56
Location   60-72
Physical effects 75   /* 1=Yes, 0=No  */
Multimedia  79  /*  1=Yes, 0=No  */
ET contact   83  /* 1=Yes, 0=No  */
Abduction   87   /*  1=Yes, 0=No  */
```

Soubor `ufocase.bat` pak obsahuje samotná data a jeho prvních pět řádků vypadá následovně:

```
1865  The Great Airships                                   USA            1   0   1  0
1878  Denison, Texas Daylight UFO                          USA            0   0   0  0
1897  The Aurora, Texas Crash                              USA            1   0   1  0
1930  Alien Picture from Alaska                            Alaska         0   1   1  0
1937  Discovery of the Dropa Discs                         China          0   0   0  0
```

Z popisku a samotných dat tedy vidíme, že se jedná o záznamy pozorování neidentifikovatelných létajících objektů (UFOs) na různých místech. K dispozici jsou ke každému případu informace o roku (`Year`), kdy k událostem došlo, písemný popisek události `Name of sighting/event`, název lokace (`Location`), kde k pozorování došlo a informace o tom, zda během pozorování došlo k nějakým fyzickým projevům (`Physical effects`), zda jsou k dispozici multimediální nahrávky z události (`Multimedia`), zda došlo ke kontaktu (`ET contact`) a zda došlo přímo k unesení mimozemšťanem (`Abduction`).

Soubor nás svojí strukturou tedy přímo vybízí k tomu, abychom z něj vytvořili `data.frame` a nějakým způsobem data analyzovali. Má to ovšem háček. Pro nás je struktura dokumentu sice jasná, ale jak to říci našemu programu? Jednotlivé sloupce jsou totiž od sebe oddělená v podstatě libovolným počtem mezer. Situaci navíc ještě komplikuje druhý sloupec s popisem události, který má mezery mezi slovy. Pojďme si tedy napsat program, který tento soubor nejprve zanalyzuje řádku po řádce a pokusí se na základě jednoduchého pravidla rozlišit záznamy z jendotlivých sloupců.

Načtení souboru v R probíhá následovně. Nejprve je třeba vytvořit připojení k souboru, ze kterého budeme jednotlivé řádky číst:

In [1]:
connection  <- file("ufocase.dat", open = "r")

Tímto způsobem tedy řekneme, že chceme orevřít soubor z daného umístění a budeme jej pouze číst (`open = "r"`). Funkce `file` umožňuje krom souborů na disku otevírat soubory i z dalších umístění a podporuje i některé běžné kompresní formáty (viz nápověda, `help(file)`). Proměnnou jsme si nazvali `connection`, protože se po technické stránce jedná o otevření připojení k určitému proudu dat, který budeme postupně číst.

Soubor máme takto tedy připravený pro čtení. V našem případě chceme soubor číst řádku po řádce, k čemuž slouží v R funkce `readlines`, které dáme k dispozici naše připojení a počet řádek, který chceme naráz ze souboru načíst:

In [2]:
readLines(connection, n = 5)

[1] "1865  The Great Airships                                   USA            1   0   1   0"
[2] "1878  Denison, Texas Daylight UFO                          USA            0   0   0   0"
[3] "1897  The Aurora, Texas Crash                              USA            1   0   1   0"
[4] "1930  Alien Picture from Alaska                            Alaska         0   1   1   0"
[5] "1937  Discovery of the Dropa Discs                         China          0   0   0   0"

Toto je tedy prvních pět řádek souboru jako seznam. Zde je dalších pět:

In [3]:
readLines(connection, n = 5)

[1] "1939  Alien Bodies Confirmed (Cordell Hull)                USA            0   0   1   0"
[2] "1941  Missouri Crash & Retrieval                           USA            1   1   1   0"
[3] "1942  The Battle of Los Angeles                            USA            1   1   0   0"
[4] "1944  World War II Foo-Fighters                            Europe         0   1   0   0"
[5] "1945  Lt. Gorman Chases a UFO                              USA            0   0   0   0"

Všimněte si, že funkce `readLines` vrací pokaždé další a další řádky. Je to tím, že objekt, který se skrývá pod proměnnou `connection`, si udržuje svůj vnitřní stav a při každém zavolání začne vracet data od té části souboru, kde dříve skončil. 

Když už připojení nepotřebujeme měli bychom ho uzavřít:

In [4]:
close(connection)

Tak zajistíme, že se připojení uzavře a není z něj již možné dál číst:

In [5]:
readLines(connection, n = 5)

ERROR: Error in readLines(connection, n = 5): invalid connection


Pokud chceme program znovu načíst, musíme tedy připojení znovu vytvořit:

In [6]:
connection <- file("ufocase.dat", open = "r")

Přistupme teď tedy již k samotné analýze souboru. Jelikož nevíme, kolik má soubor řádek musíme použít cyklus `while` nebo `repeat`. My zde použijeme cyklus `repeat`, který v tomto případě umožňuje použít o něco čitelnější kód:

In [7]:
counter <- 1
repeat {
    line <- readLines(connection, n = 1)
    if (!length(line)) break
    print(line)
}

[1] "1865  The Great Airships                                   USA            1   0   1   0"
[1] "1878  Denison, Texas Daylight UFO                          USA            0   0   0   0"
[1] "1897  The Aurora, Texas Crash                              USA            1   0   1   0"
[1] "1930  Alien Picture from Alaska                            Alaska         0   1   1   0"
[1] "1937  Discovery of the Dropa Discs                         China          0   0   0   0"
[1] "1939  Alien Bodies Confirmed (Cordell Hull)                USA            0   0   1   0"
[1] "1941  Missouri Crash & Retrieval                           USA            1   1   1   0"
[1] "1942  The Battle of Los Angeles                            USA            1   1   0   0"
[1] "1944  World War II Foo-Fighters                            Europe         0   1   0   0"
[1] "1945  Lt. Gorman Chases a UFO                              USA            0   0   0   0"
[1] "1947  The Kenneth Arnold Sighting                      

Takto je tedy celý náš datový soubor vypsaný řádku po řádce. Všimněte si, že výstup funkce `length` lze přímo použít jako pravdivostní hodnotu. Pokud totiž ze souboru vyčerpáme všechen vstup, funkce nám vrátí prázdný vektor:

In [8]:
print(readLines(connection, n = 1))

character(0)


Jehož délka je tedy nulová:

In [9]:
length(character(0))

[1] 0

Operátor `!` se sice většinou aplikuje na logické hodnoty, ale lze jej použít i na čísla:

In [10]:
!-2
!-1
!0
!1
!2

[1] FALSE

[1] FALSE

[1] TRUE

[1] FALSE

[1] FALSE

Každou nenulovou hodnotu tak tento operátor interpretuje jako `TRUE`, a tudíž po negaci vrátí `FALSE`. Této vlastnosti jsme s výhodou využili i v našem kódu jako rozhdodovací podmínku k zastavení cyklu:

```R
if (!length(line)) break
```

Jsme tedy hotoví a na závěr ještě uzavřeme připojení:

In [11]:
close(connection)

Pro úplnost si ještě uvedeme implementaci s cyklem `while`:

In [12]:
connection <- file("ufocase.dat", open = "r")

counter <- 1
while (length(line <- readLines(connection, n = 1))) {
    print(line)
    
    # ukončení po prvních pěti řádcích, aby nebyl výstup příliš dlouhý
    if (counter >= 5) break
    counter <- counter + 1
}

close(connection)

[1] "1865  The Great Airships                                   USA            1   0   1   0"
[1] "1878  Denison, Texas Daylight UFO                          USA            0   0   0   0"
[1] "1897  The Aurora, Texas Crash                              USA            1   0   1   0"
[1] "1930  Alien Picture from Alaska                            Alaska         0   1   1   0"
[1] "1937  Discovery of the Dropa Discs                         China          0   0   0   0"


Zde jsme se rozhodli přiřazení do proměnné `line` a přečtení řádky z připojení uskutečnit přímo při definici podmínky pro provedení další iterace. V R je toto možné a i když je to méně čitelné než řešení nahoře, působí to velice kompaktně. My zde cyklus ukončíme předčasně po pěti řádcích, ale je vám asi jasné, že byste takto opět mohli projít celý soubor.

Vypsali jsme si teď tedy celý soubor, který naštěstí není tak dlouhý, a vidíme, že sloupce, které představují jednotlivé nezávislé proměnné, jsou většinou oddělené alespoň dvěma mezerami. Zkusme tedy za použití této znalosti nejprve vytvořit jednoduchou matici, která bude reprezentovat naše data:

In [13]:
connection <- file("ufocase.dat", open = "r")

counter <- 1
m_ufocase <- NULL # proměnná, která bude odkazovat na naší matici
while (length(line <- readLines(connection, n = 1))) { # iteruj řádku po řádce
    line_strip <- unlist(strsplit(line, " ")) # rozděl řádku podle mezer
    
    data_row <- c() # vektor reprezentující jeden řádek v matici
    next_col <- T # indikátor, zda očekáváme nový sloupec (nová nezávislá proměnná)
    for (fragment in line_strip) { # iteruj přes jednotlivé nalezené fragmenty
        if (fragment != "" && next_col) {
            # nalezený fragment není prázdný řetězec a zároveň očekáváme nový sloupec
            
            data_row <- c(data_row, fragment) # přidej nové slovo na řádku v matici
            next_col <- F # nalezli jsme sloupec, takže tento fakt zaznamenáme
        } else if (fragment == "") {
            # nalezli jsme prázdý řetězec, což znamená úsek, 
            # kde byla v textové tabulce na řádce více než jedna mezera
            
            next_col <- T # očekáváme tedy nový sloupec
        } else if (fragment != "" && !next_col) {
            # nalezli jsme fragment, ale neočekáváme nový sloupec
            # jedná se tedy o fragment z jednoho sloupce
            
            # z řádku si tedy vezmeme poslední fragment
            previous_fragment <- data_row[length(data_row)]
            
            # a po spojení s předchozím tento celek přidáme jako nový poslední fragment
            data_row <- c(
                data_row[-(length(data_row))] # všechna data na řádce ale bez posledního fragmentu
                , paste(previous_fragment, fragment, sep = " ") # spojení posledního fragmentu s aktuálním
            )
        } else {
            stop("Sem bychom se neměli dostat. Pokud ano, něco nám uniklo.")
        }
    }
    
    # připoj aktuální řádek do matice
    m_ufocase <- rbind(m_ufocase, data_row)
}

close(connection) # zavři spojení

head(m_ufocase, 5) # ukaž prvních pět řádků z vytvořené matice 

data_row,1865,The Great Airships,USA,1,0,1,0
data_row,1878,"Denison, Texas Daylight UFO",USA,0,0,0,0
data_row,1897,"The Aurora, Texas Crash",USA,1,0,1,0
data_row,1930,Alien Picture from Alaska,Alaska,0,1,1,0
data_row,1937,Discovery of the Dropa Discs,China,0,0,0,0


Jak vidíte, připravit si data k analýze může někdy vyžadovat nemalé úsilí. V tomto případě jsme museli přijít s poměrně rozvětveným algoritmem, abychom data dostali do podoby, se kterou již lze snáze pracovat. V předchozí ukázce je několik nových funkcí, které jsme ještě nediskutovali. Pojďme si je tedy teď probrat podrobněji.

**Funkce `stop`**

Funkce stop je podobná jako funkce `warning` a znamená, že se aktuálně vykonávaný program ukončí a zobrazí uživateli varovnou zprávu o chybě:

In [14]:
stop("Varování!")

ERROR: Error in eval(expr, envir, enclos): Varování!


Pokud je zavolání funkce stop uzavřeno v nějakém bloku kódu, ukáže se i konkrétní řádka programu, kde k zavolání došlo:

In [15]:
{
    print("OK")
    stop("Varování!")
    print("Not OK")
}

[1] "OK"


ERROR: Error in eval(expr, envir, enclos): Varování!


Pokud k volání dojde ve funkci, je součástí výpisu i výpis toho, v jaké fázi se zrovna program nacházel:

In [16]:
x <- function () stop("Not OK.")
x()

ERROR: Error in x(): Not OK.


To zde tedy znamená informaci o tom, že nejdříve došlo k zavolání funkce `x`, a potom k zastavení programu funkcí `stop`.

**Funkce `rbind`**

Funkce `rbind` je velice užitečnou funkcí pro práci s daty a slouží ke spojování dvou datových reprezentací se stejným počtem sloupců. Lze takto například spojit dvě matice:

In [17]:
m_x <- matrix(rep(0, 6), ncol = 2)
m_y <- matrix(rep(1, 6), ncol = 2)

rbind(m_x, m_y)

0,0
0,0
0,0
1,1
1,1
1,1


nebo i jiné vzájemně kompatibilní struktury, kterých může být libovolný počet:

In [18]:
rbind(c(1,2), c(3,4), c(5, 6))

1,2
3,4
5,6


Když je jediným argumentem vektor, je výsledkem jednořádková matice:

In [19]:
print(rbind(1:10))

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]    1    2    3    4    5    6    7    8    9    10


čehož jsme využili i v našem kódu pro zpracování textového souboru.

Příbuznou funkcí k `rbind`, je `cbind`, která naopak sdružuje data přes sloupce:

In [20]:
cbind(m_x, m_y)
cbind(c(1,2), c(3,4), c(5, 6))

0,0,1,1
0,0,1,1
0,0,1,1


1,3,5
2,4,6


**Funke `head`**

Tato funkce se hodí zejména pokud pracujete v interaktivním módu a chcete se podívat, jak data vlastně vypadají, ale bez toho aniž byste museli vypisovat úplně všechno. Funkci můžete též specifikovat kolik položek maximálně chcete zobrazit. U dvourozměrných datových struktur je to počet řádek:

In [21]:
head(m_ufocase, 3)

data_row,1865,The Great Airships,USA,1,0,1,0
data_row,1878,"Denison, Texas Daylight UFO",USA,0,0,0,0
data_row,1897,"The Aurora, Texas Crash",USA,1,0,1,0


U vektorů potom pouze počet položek:

In [22]:
head(1:10, 5)

[1] 1 2 3 4 5

Mimo funkce `head` existuje ještě funkce `tail`, která dělá opět to samé, ale od konce:

In [23]:
tail(m_ufocase, 3)

data_row,2004,"Centerville, Ohio UFO Report",USA,1,0,0,0
data_row,2004,Close Encounter in Tennessee,USA,1,0,0,0
data_row,2004,Northern Ohio Sightings,USA,1,0,0,0


## Funkce `apply`, `lapply` a `sapply`

Vraťme se teď k našemu datovému souboru s pozorováním UFO. Asi jste si všimli, že funkce `rbind` automaticky přidala jména řádků podle názvu připojované proměnné. V našem případě nás to jen zbytečně rozptyluje, takže tato jména z matice odstraníme, než začneme s dalšími úpravami:

In [24]:
row.names(m_ufocase) <- NULL
head(m_ufocase)

1865,The Great Airships,USA,1,0,1,0
1878,"Denison, Texas Daylight UFO",USA,0,0,0,0
1897,"The Aurora, Texas Crash",USA,1,0,1,0
1930,Alien Picture from Alaska,Alaska,0,1,1,0
1937,Discovery of the Dropa Discs,China,0,0,0,0
1939,Alien Bodies Confirmed (Cordell Hull),USA,0,0,1,0


Všechna data v matici jsou zatím typu `character`:

In [25]:
class(m_ufocase)
mode(m_ufocase)

[1] "matrix"

[1] "character"

Je však zřejmé, že některá data by bylo vhodné reprezentovat i jinými datovými typy. Všechny položky (kromě textového popisu události) by vlastně bylo vhodné reprezentovat pomocí faktorů, neboť řadí jednotlivé záznamy do různých skupin. Vhodnější reprezentací by tedy byl `data.frame`, který je rozšířením seznamů a umožňuje tak agregovat proměnné různých typů. 

V následujících kapitolách si postupně představíme funkce, které nám mohou pomoci nejen při převodu naší matice, ale i při řešení dalších problémů. Funkce, které si teď ukážeme, znamenají v R totiž jednu z funkcionálních alterantiv k cyklům a umožňují rychlé procházení napříč položkami nejrůznějších datových struktur a jejich transformace. První takovou funkcí je `apply`, která je z výše uvedených nejjednodušší  zpravidla se aplikuje na matice.

### `apply`

Jednoduchou aplikaci funkce `apply`, představuje následující příklad:

In [26]:
connect <- function(x) {
    paste(x, sep = '', collapse = '--')
}

output <- apply(m_ufocase, 1, connect)
output[1:5]

[1] "1865--The Great Airships--USA--1--0--1--0"            
[2] "1878--Denison, Texas Daylight UFO--USA--0--0--0--0"   
[3] "1897--The Aurora, Texas Crash--USA--1--0--1--0"       
[4] "1930--Alien Picture from Alaska--Alaska--0--1--1--0"  
[5] "1937--Discovery of the Dropa Discs--China--0--0--0--0"

V tomto příkladě funkce `apply` funguje obdobně jako následující cyklus:

In [27]:
output <- NULL
for (row_idx in 1:nrow(m_ufocase)) {
    # pro každý řádek
    x <- m_ufocase[row_idx,]
    
    # zavolej funkci `connect`, kde její první argument bude právě procházený řádek matice
    output <- c(output, connect(x)) 
}

output[1:5]

[1] "1865--The Great Airships--USA--1--0--1--0"            
[2] "1878--Denison, Texas Daylight UFO--USA--0--0--0--0"   
[3] "1897--The Aurora, Texas Crash--USA--1--0--1--0"       
[4] "1930--Alien Picture from Alaska--Alaska--0--1--1--0"  
[5] "1937--Discovery of the Dropa Discs--China--0--0--0--0"

První argument funkce `apply` tedy specifikuje matici, kterou budeme procházet. Druhý argument je potom kód dimenze (1 pro řádky, 2 pro sloupce, 3 pro třetí dimenzi atd.). Třetí argument pak specfikuje funkci, která se bude volat a musí mít alespoň jeden argument. Řádek nebo sloupec se vždy pošle jako první argument funkce. Výsledky jednotlivých volání jsou pak spojené do jednoho vektoru, popř. matice u složitějších datových struktur:

In [28]:
transpose <- function(x) {
    as.matrix(x, nrow=length(x))
}

output <- apply(m_ufocase, 1, transpose)
output

1865,1878,1897,1930,1937,1939,1941,1942,1944,1945,⋯,2002,2003,2003,2003,2003,2004,2004,2004,2004,2004
The Great Airships,"Denison, Texas Daylight UFO","The Aurora, Texas Crash",Alien Picture from Alaska,Discovery of the Dropa Discs,Alien Bodies Confirmed (Cordell Hull),Missouri Crash & Retrieval,The Battle of Los Angeles,World War II Foo-Fighters,Lt. Gorman Chases a UFO,⋯,"Possible UFO Landing, Chile",UFO Photographed / Wisconsin,"UFO Video / Tennessee, Shaffer","UFO Incident, Screaming Sounds","Ashland, Nebraska UFO Video","Amelia, Ohio Bizarre UFO Incident",Mexican Military Releases UFO Footage,"Centerville, Ohio UFO Report",Close Encounter in Tennessee,Northern Ohio Sightings
USA,USA,USA,Alaska,China,USA,USA,USA,Europe,USA,⋯,Chile,USA,USA,USA,USA,USA,Mexico,USA,USA,USA
1,0,1,0,0,0,1,1,0,0,⋯,1,0,0,1,0,1,1,1,1,1
0,0,0,1,0,0,1,1,1,0,⋯,1,1,1,0,1,0,1,0,0,0
1,0,1,1,0,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


Funkci lze kromě dat samotných též poslat další argumenty, ty ve volání následují hned po funkci samotné a zůstavají při každém volání konstantní:

In [29]:
connect <- function(x, sep = '', collapse = '--') {
    paste(x, sep = sep, collapse = collapse)
}

output <- apply(m_ufocase, 1, connect, collapse = '----')
output[1:5]

[1] "1865----The Great Airships----USA----1----0----1----0"            
[2] "1878----Denison, Texas Daylight UFO----USA----0----0----0----0"   
[3] "1897----The Aurora, Texas Crash----USA----1----0----1----0"       
[4] "1930----Alien Picture from Alaska----Alaska----0----1----1----0"  
[5] "1937----Discovery of the Dropa Discs----China----0----0----0----0"